<a href="https://colab.research.google.com/github/SayantikaFSU/ocp-ci-analysis/blob/master/PlantPathology_imageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'plant-pathology-2021-fgvc8:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F25563%2F2094376%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240518%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240518T154615Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D930090ea8664030c5d80fa83b29236ffc8218d68b169b625235e720a1b4cdb4e2e1cbead926a389c0adc1cb2be0afab7011d6e66496362f4e4e94e6487473118f7b6d568b6b43e150dba9734800882323e5a5e67e68b89575b68311d2f24b9f076490c86fa05c927491699baa4d9bc052880a7af9f65ee4daa36c3fed2f97d5e0733c350c8d072d812df8360a54522926470d1c0eae2b519d4dcf41bcddb2cb4e56984c5b7857e89e504948089116a14c8aa7b8603cb1fe0e7a4b879177807999bc1df7bd7b6e3625ae52b5eb976876a99240ebadd8c7bfce2199cd585a1ae7cccad36a7d86299da70328e5c1a5deba6537e9995f6867ab8ae2fd11b3862a3a2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as Fn
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from scipy.io import savemat
from time import time
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import os
import torchvision
import pandas as pd

from PIL import Image


import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

INTRODUCTION:

This notebook deals with Classification of the disease of plants using the 'Plant Pathology 2021 - FGVC8' Dataset.

The goal of this notebook is to train a neural network on the train-image dataset and used that trained network to correctly predict the disease of the test images.

**********************************************************************************************

In [ ]:
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.10-cp37-cp37m-linux_x86_64.whl


In [ ]:
#import torch

# Set device to TPU
#device = xm.xla_device()


DATASETS PROVIDED:


Here the image data is provided in two directories

(1) train_images : contains 18632 .jpg images

(2) test_images : contains 3  .jpg images

(3) train.csv : a csv file containing two columns providing the train image filename and the disease labels

************************************************************************************

In [ ]:
train_path='/kaggle/input/plant-pathology-2021-fgvc8/train.csv'

train_dir ='/kaggle/input/plant-pathology-2021-fgvc8/train_images'
test_dir='/kaggle/input/plant-pathology-2021-fgvc8/test_images'

train=pd.read_csv(train_path)


print(f'Number of images = {len(train)}')

train.head(4)

In [ ]:
# Plotting one image

i=0

name_img = train.iloc[i, 0]
label_name_img=train.iloc[i,1]

img = Image.open(os.path.join(train_dir, name_img[:-4]+".jpg"))
plt.imshow(img)
plt.title(label_name_img)

The images are seen to be really big in size so I transformed them into smaller 128 x 128 size to handle easily

In [ ]:
img=np.array(img)
img.shape

CATEGORIES OF DISEASE:

We can observe there are 12 categories of disease.

Although the count plot shows that there exists an imbalance between the categories, so usual cross-entropy loss would not be a good fit for the loss function, since it is biased towards the dominant classes.

********************************************************************************************

In [ ]:
# plot of label categories

# Count the occurrences of each category
category_counts = train['labels'].value_counts()
print(category_counts)
print(f"number of categories={len(category_counts)}")

category_counts.index

In [ ]:
train['labels'].value_counts().plot(kind='bar')
plt.suptitle('Count Plot of different disease category')

IMAGES of leaves belonging to DIFFERENT CATEGORIES

************************************************************

In [ ]:
# Function to display images belonging to a class:

def show_image(class_name, examples):
    image_list=train[train['labels']==class_name]['image'][0:examples].to_list()


    plt.figure(figsize=(20, 10))

    for i in range(0,len(image_list)):
        name_img = image_list[i]
        img = Image.open(os.path.join(train_dir, name_img[:-4]+".jpg"))
        img.resize ((128,128),resample=Image.BICUBIC)


        plt.subplot(1 ,examples, i%examples +1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis('off')


#show_image('healthy',3)


In [ ]:
class_list=['scab', 'healthy', 'frog_eye_leaf_spot', 'rust', 'complex',
       'powdery_mildew', 'scab frog_eye_leaf_spot',
       'scab frog_eye_leaf_spot complex', 'frog_eye_leaf_spot complex',
       'rust frog_eye_leaf_spot', 'rust complex', 'powdery_mildew complex']

for k in range(0,len(class_list)):
    class_name=class_list[k]
    show_image(class_name,2)

MODELS CONSIDERED:
****************************************************************

Data Transformation:

Since the images are really big in size I used the following function to reduce them to 128 x 128 and also transform them into tensors


In [ ]:
#transform all the training data into tensors

from torch.utils.data import DataLoader

import os
import torch
from torchvision import transforms,datasets

from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, RandomCrop,ToTensor, Normalize, Grayscale, RandomRotation,InterpolationMode
from torchvision.transforms.v2 import  RandomResize

# Define data transformations
transform = Compose([
    Resize([128, 128],interpolation=InterpolationMode.BICUBIC),
    ToTensor()
    ])

MODEL 1:

The first model I considered is a CNN model with four layers described as below

*********************************************************************************************

CNN model:

layer 1 :conv-relu-maxpool-dropout

layer 2 :conv-relu-maxpool-dropout

layer 3 :conv-relu-maxpool-dropout

layer 4 :conv-relu-maxpool-dropout

*********************************************************************************



CUSTOM DATASET to preprocess the data for the model
*********************************************************

train.csv : contains the image names and labels

train_dir : contains the images

Custom Dataset returns the image_tensor and the corresponding text_labels

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class CustomTextLabelDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.csv_path = csv_path
        self.images_root = image_dir
        self.transform = transform
        self.data = self.load_data()

    def load_data(self):
        # Load data from the CSV file
        df = pd.read_csv(self.csv_path)

        # Combine filename and label information
        data = [{'filename': row['image'], 'label': row['labels']} for _, row in df.iterrows()]

        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        entry = self.data[index]
        image_path = os.path.join(self.images_root, entry['filename'])
        text_label = entry['label']

        # Load image
        image = Image.open(image_path)

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, text_label


In [ ]:
# Specify the path to CSV file and images directory:
csv_path = '/kaggle/input/plant-pathology-2021-fgvc8/train.csv'
images_root = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'

## dataset:
full_dataset = CustomTextLabelDataset(csv_path=csv_path, image_dir=images_root, transform=transform)



In [ ]:
#access label
full_dataset[0][1]

LOSS FUNCTION: FOCAL LOSS
*******************************************************

Since we have a pretty significant imbalance in the category counts the classic cross entropy loss would show that less frequent classes will having a more significant loss.


Focal Loss is a modification of the standard cross-entropy loss, designed to address the problem of class imbalance in binary and multi-class classification tasks.
The focal loss is a finetuned cross-entropy loss that puts emphasis on classes with a high loss and give less importance to classes with low loss. This regularisation term pushes the model to focus on learning rare classes and put less emphasis on dominant classes.
It was introduced in the paper titled "Focal Loss for Dense Object Detection" by Tsung-Yi Lin, Priya Goyal, Ross Girshick, Kaiming He, and Piotr Dollár.

The Focal Loss (FL) is defined as follows:

https://doi.org/10.48550/arXiv.1708.02002

********************************
The CNN model with four layers explained earlier


In [ ]:
import torch
import torch.nn as nn

class CNN_model(nn.Module):
    def __init__(self, num_classes):
        super(CNN_model, self).__init__()

        #### 1st Convolutional Layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu1 = nn.ReLU()
        # Max-pooling layer
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        #### 2nd Convolutional Layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu2 = nn.ReLU()
        # Max-pooling layer
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #### 3rd Convolutional Layer
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu3 = nn.ReLU()
        # Max-pooling layer
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        #### 4th Convolutional Layer
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu4 = nn.ReLU()
        # Max-pooling layer
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        ### Fully Connected Layer
        self.fc = nn.Linear(128 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = self.maxpool4(self.relu4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


num_classes = 12  # we have 12 classes here
model = CNN_model(num_classes)

model


An IMPLEMENTATION OF FOCAL LOSS function using Pytorch:
******************************************

here,

alpha = balancing factor to handle the class imbalance computed sample wise

gamma =  a focusing parameter that allows the model to down-weight easy examples

alpha and gamma are hyperparameters that can be adjusted based on specific problem and dataset. The reduction parameter specifies how the losses from individual samples should be aggregated.

Returns average focul loss from all the samples

In [ ]:

import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        #focul loss
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha*(1 - pt) ** self.gamma * ce_loss

        #if self.alpha is not None:
            #alpha_factor = self.alpha[targets]
            #focal_loss = focal_loss * alpha_factor


        # Apply reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        elif self.reduction == 'none':
            return focal_loss
        else:
            raise ValueError("Invalid reduction option. Use 'mean', 'sum', or 'none'.")


Since the labels are given in text format I made a mapping to turn text into numerics and vice-versa
*********************

In [ ]:
# Create a mapping between text labels and numerical indices
unique_text_labels = class_list
label_to_index = {label: index for index, label in enumerate(unique_text_labels)}
index_to_label = {index: label for label, index in label_to_index.items()}


TRAINING ON train_images:
*************************************************************************************************
Used Adam optimizer with learning rate 0.01.


the model is saved locally at 'cnn_model.pth'

In [ ]:
from torch.utils.data import DataLoader, random_split
#from torch.optim.lr_scheduler import StepLR

# Define the focal loss function and optimizer:
criterion = FocalLoss(gamma=2, alpha=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
#step_scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Wrap model and optimizer with XLA
#model, optimizer = xm.initialize_model(model, optimizer, opt_level="O2")


# Split the dataset into training and validation sets: 80:20
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
print(len(train_dataset))

# Create data loaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print(len(train_loader))


# Use GPU for model
model.to(device)


epochs = 10

for epoch in range(epochs):

    ## Training loop:

    model.train()  # Set the model to training mode

    #run=0
    for inputs, text_labels in train_loader:
        inputs, text_labels = inputs.to(device), text_labels

        #print('input',inputs)
        #print('text_labels',text_labels)
        #print(inputs.shape)

        # Convert text labels to numerical indices using the mapping
        numerical_labels = [label_to_index[label] for label in text_labels]
        #print((numerical_labels))

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        #print(outputs.shape)
        #print(torch.tensor(numerical_labels).shape)

        loss=criterion(outputs, torch.tensor(numerical_labels).to(device))  # Focal Loss


        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        #print(f'{run}run done')
        #run=run+1

    #print(f'train loss for {epoch+1} epoch = {loss:0.3f} ')

    ###Validation loop:

    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, text_labels in val_loader:
            inputs, text_labels = inputs.to(device), text_labels


            numerical_labels = [label_to_index[label] for label in text_labels]
            numerical_labels=torch.tensor(numerical_labels).to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += len(numerical_labels)
            correct += (predicted == numerical_labels).sum().item()

            #step_scheduler.step()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Train Set Loss: {loss.item():.4f}, Validation Set Accuracy: {accuracy:.4f}')




In [ ]:
## Save the trained model:
torch.save(model.state_dict(), 'cnn_model.pth')


PREDICTION ON TEST IMAGES:

*****************************************************************************

In [ ]:
### On test data:

#Step 1:

# Load the saved model state dict
model.load_state_dict(torch.load('/kaggle/working/cnn_model.pth'))

# Set the model to evaluation mode
model.eval()

In [ ]:


# Step 2: Prepare Test Data
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images'

os.listdir(test_dir)

In [ ]:
for filename in os.listdir(test_dir):
    img_path=os.path.join(test_dir,filename)
    img=Image.open(img_path)
    plt.imshow(img)
    plt.show()
    img_tensor=transform(img).unsqueeze(0)
    #print(img_tensor.shape)

    with torch.no_grad():

        pred=model(img_tensor.to(device))
        #print(pred.shape)

        predicted_class = torch.argmax(pred).item()
        predicted_text_class=index_to_label[predicted_class]
        print(f"Image: {filename}, Predicted Class: {predicted_text_class}")


Model 2:

CNN:

4 layers like Model 1 with a dropout at the end

****************************************************************************************

In [ ]:
import torch
import torch.nn as nn

class CNN_model_dropout(nn.Module):
    def __init__(self, num_classes):
        super(CNN_model_dropout, self).__init__()

        #### 1st Convolutional Layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu1 = nn.ReLU()
        # Max-pooling layer
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.25)

        #### 2nd Convolutional Layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu2 = nn.ReLU()
        # Max-pooling layer
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(0.25)


        #### 3rd Convolutional Layer
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu3 = nn.ReLU()
        # Max-pooling layer
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout3 = nn.Dropout(0.25)

        #### 4th Convolutional Layer
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        # ReLU activation
        self.relu4 = nn.ReLU()
        # Max-pooling layer
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout4 = nn.Dropout(0.25)

        ### Fully Connected Layer
        self.fc = nn.Linear(128 * 8 * 8, num_classes)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))
        x = self.maxpool4(self.relu4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


num_classes = 12  # we have 12 classes here
model = CNN_model(num_classes)

model

In [ ]:
from torch.utils.data import DataLoader, random_split
#from torch.optim.lr_scheduler import StepLR

# Define the focal loss function and optimizer:
criterion = FocalLoss(gamma=2, alpha=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
#step_scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

# Wrap model and optimizer with XLA
#model, optimizer = xm.initialize_model(model, optimizer, opt_level="O2")


# Split the dataset into training and validation sets: 80:20
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
print(len(train_dataset))

# Create data loaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print(len(train_loader))


# Use GPU for model
model.to(device)


epochs = 8

for epoch in range(epochs):

    ## Training loop:

    model.train()  # Set the model to training mode

    #run=0
    for inputs, text_labels in train_loader:
        inputs, text_labels = inputs.to(device), text_labels

        #print('input',inputs)
        #print('text_labels',text_labels)
        #print(inputs.shape)

        # Convert text labels to numerical indices using the mapping
        numerical_labels = [label_to_index[label] for label in text_labels]
        #print((numerical_labels))

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        #print(outputs.shape)
        #print(torch.tensor(numerical_labels).shape)

        loss=criterion(outputs, torch.tensor(numerical_labels).to(device))  # Focal Loss


        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        #print(f'{run}run done')
        #run=run+1

    #print(f'train loss for {epoch+1} epoch = {loss:0.3f} ')

    ###Validation loop:

    model.eval()  # Set to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, text_labels in val_loader:
            inputs, text_labels = inputs.to(device), text_labels


            numerical_labels = [label_to_index[label] for label in text_labels]
            numerical_labels=torch.tensor(numerical_labels).to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += len(numerical_labels)
            correct += (predicted == numerical_labels).sum().item()

            #step_scheduler.step()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{epochs}, Train Set Loss: {loss.item():.4f}, Validation Set Accuracy: {accuracy:.4f}')



## Save the trained model:
torch.save(model.state_dict(), 'cnn_model_dropout.pth')


In [ ]:
### On test data:

#Step 1:

# Load the saved model state dict
model.load_state_dict(torch.load('/kaggle/working/cnn_model_dropout.pth'))

# Set the model to evaluation mode
model.eval()

In [ ]:
for filename in os.listdir(test_dir):
    img_path=os.path.join(test_dir,filename)
    img=Image.open(img_path)
    plt.imshow(img)
    plt.show()
    img_tensor=transform(img).unsqueeze(0)
    #print(img_tensor.shape)

    with torch.no_grad():

        pred=model(img_tensor.to(device))
        #print(pred.shape)

        predicted_class = torch.argmax(pred).item()
        predicted_text_class=index_to_label[predicted_class]
        print(f"Image: {filename}, Predicted Class: {predicted_text_class}")
